In [19]:
!pip install pyspark


عبد الرحمن شوف
أول شي انبه عليه الجلسة للمنصة هنا تبدأ تسجل عالموقع لما تشغل نت ولما يكون فيه انترنت لكن بدون نت نقدر نستخدمه ككتابة اكواد لكن ما ينحفظو فانتبه 

هلقيت بدي أشرحلك ليش كتبت كل كود عشان تفهم وين أنا واصل وتكمل معي تطبيق المشروع


الآن هذا الكود الي فوق
يعني بنستخدم هذا السطر عشان نثبت الباي سبارك على كاجل
 
لأنه ما بتنزل مكتبة الباي سبارك بشكل افتراضي فهذا الكود لازم لنثبتها فالسيشن تاعنا الي هنشتغل عليه يديويا كما فالكود الأول لتنزيل المكتبة بشكل يدوي

وأفهمك مكتبة الباي سبارك هي الواجهة الي بتخلينا نستخدم سبارك هي تشبه عمل الاباتشي سبارك الي معنا لكن هذه الاداة بلغة البايثون وبدونها ما نقدر نبدأ جلسة السبارك ولا نشغل اوامر سبارك بجيبلنا ايرور فهذا السطر ضروري يشتغل عنعا فالجلسة

The pyspark library must be downloaded before starting the rapid and parallel data analysis process using Spark.


In [20]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("Cloud-Based DDoS Analysis Platform") \
    .master("local[*]") \
    .getOrCreate()


هختصر طريق واقلك عشان شو اعملت كل كود فبعد كل كود هتشوف كلمة #عشان الي هي انت فاهمها يعني نفس لسبب ومن هنا هبدأ


عشان بدنا نبدأ الجلسة لازم نكتب هذا الكود كما في تعليمات موقع السبارك لبداية السيششن او الجلسة للسبارك
الكود كله عبارة عن بلوك مش ضروري تفهمه مع انه من الكلمات لوحده ينفهم مثل انه السبارك بنى تطبيق وسماه جوا السلاش نكتب اي اسم بدنا اياه وبعديها اعطناه صلاحية الوصول لكل البيانات محليا وبعدها قلتله يا بتاخد الجلسة لو كانت معمولة يا بتنشئها لو لسا لاول مرة هتعملها

To start a session, we need to write this code as per the instructions on the Spark Apache website for starting a session. The entire code is a block; you don't necessarily need to understand it, although it's understandable from the words themselves. For example, Spark built an application and named it within the slash. We write any name we want, then we give it permission to access all local data. After that, I tell it to either take the session if it already exists or create it if it's the first time you're doing this.

In [21]:
from pyspark.sql.functions import col, isnan, when, count, mean, min, max, stddev
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# ==========================
#        دوال الإحصاء
# ==========================
def dataset_info():
    global df
    return f"Rows: {df.count()} | Columns: {len(df.columns)}"

def missing_values():
    global df
    rows = df.count()
    stats = df.select([
        (count(when(col(c).isNull() | isnan(c), c)) / rows * 100).alias(c)
        for c in df.columns
    ])
    return stats.toPandas().T

def descriptive_stats():
    global df
    numeric_cols = [c for c, t in df.dtypes if t in ('int', 'double', 'float', 'long')]
    stats = df.select(
        [mean(c).alias(c+"_mean") for c in numeric_cols] +
        [min(c).alias(c+"_min") for c in numeric_cols] +
        [max(c).alias(c+"_max") for c in numeric_cols] +
        [stddev(c).alias(c+"_std") for c in numeric_cols]
    )
    return stats.toPandas().T

def average_numeric_columns():
    global df
    numeric_cols = [c for c, t in df.dtypes if t in ('int', 'double', 'float', 'long')]
    averages = df.select([mean(c).alias(c) for c in numeric_cols])
    return averages.toPandas().T


In [22]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.regression import LinearRegression
from pyspark.ml.clustering import KMeans
from pyspark.ml.fpm import FPGrowth

# دالة تحويل كل الأعمدة النصية لرقمية
def prepare_features(df, target_col=None):
    string_cols = [c for c, t in df.dtypes if t == "string" and c != target_col]
    numeric_cols = [c for c, t in df.dtypes if t in ["int", "double", "float", "long"]]

    indexers = [StringIndexer(inputCol=c, outputCol=f"{c}_indexed", handleInvalid="skip") for c in string_cols]
    feature_cols = numeric_cols + [f"{c}_indexed" for c in string_cols]

    assembler = VectorAssembler(inputCols=feature_cols, outputCol="features", handleInvalid="skip")
    pipeline = Pipeline(stages=indexers + [assembler])

    df_prepared = pipeline.fit(df).transform(df)
    return df_prepared, feature_cols



def run_decision_tree(df, label_col):
    if label_col not in df.columns:
        return f"Column '{label_col}' not found"

    feature_cols = [
        c for c, t in df.dtypes
        if t in ("int", "double", "float", "long") and c != label_col
    ]

    if len(feature_cols) < 1:
        return "Not enough numeric features"

    indexer = StringIndexer(
        inputCol=label_col,
        outputCol="label",
        handleInvalid="skip"
    )

    df_indexed = indexer.fit(df).transform(df)

    assembler = VectorAssembler(
        inputCols=feature_cols,
        outputCol="features",
        handleInvalid="skip"
    )

    df_final = assembler.transform(df_indexed).select("features", "label")

    train_df, test_df = df_final.randomSplit([0.7, 0.3], seed=42)

    dt = DecisionTreeClassifier(
        labelCol="label",
        featuresCol="features",
        maxDepth=4
    )

    model = dt.fit(train_df)
    preds = model.transform(test_df)

    evaluator = MulticlassClassificationEvaluator(
        labelCol="label",
        predictionCol="prediction",
        metricName="accuracy"
    )

    accuracy = evaluator.evaluate(preds)

    path = f"/kaggle/working/decision_tree_{label_col}.csv"
    preds.select("prediction", "label").toPandas().to_csv(path, index=False)

    return f"Decision Tree Accuracy ({label_col}): {accuracy:.4f}\n Saved to {path}"




# -------------------------
# Linear Regression
# -------------------------
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.sql.functions import col

def run_linear_regression(df, target_col):
    # التأكد أن العمود موجود
    if target_col not in df.columns:
        return f"Column '{target_col}' not found in dataset"

    # اختيار كل الأعمدة العددية ما عدا العمود الهدف كـ features
    numeric_cols = [c for c, t in df.dtypes if t in ('int', 'double', 'float', 'long') and c != target_col]

    if not numeric_cols:
        return "Not enough numeric features for Linear Regression"

    assembler = VectorAssembler(inputCols=numeric_cols, outputCol="features", handleInvalid="skip")
    df_features = assembler.transform(df).select("features", col(target_col).alias("label"))

    lr = LinearRegression(featuresCol="features", labelCol="label")
    model = lr.fit(df_features)

    preds = model.transform(df_features)

    evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
    rmse = evaluator.evaluate(preds)

    # حفظ النتائج
    path = "/kaggle/working/linear_regression_predictions.csv"
    preds.select("prediction", "label").toPandas().to_csv(path, index=False)

    return f"Linear Regression RMSE: {rmse:.4f}\n Saved to: {path}"


# -------------------------
def run_kmeans(df, k=3):
    # نختار فقط الأعمدة الرقمية
    feature_cols = [
        c for c, t in df.dtypes
        if t in ('int', 'double', 'float', 'long')
    ]

    if len(feature_cols) < 2:
        return " Not enough numeric columns for KMeans"

    # تجميع الخصائص
    assembler = VectorAssembler(
        inputCols=feature_cols,
        outputCol="features",
        handleInvalid="skip"
    )

    df_features = assembler.transform(df).select("features")

    # تدريب KMeans
    kmeans = KMeans(
        k=k,
        seed=42,
        featuresCol="features"
    )

    model = kmeans.fit(df_features)
    preds = model.transform(df_features)

    # حفظ النتائج
    path = "/kaggle/working/kmeans_clusters.csv"
    preds.select("prediction").toPandas().to_csv(path, index=False)

    return (
        f" KMeans clustering completed\n"
        f" Number of clusters: {k}\n"
        f" Saved to {path}"
    )


# -------------------------
# FPGrowth Frequent Itemsets
# -------------------------
from pyspark.sql.functions import array, col
from pyspark.sql.types import StringType

def run_fpgrowth(df, item_col):
    # التأكد أن العمود موجود
    if item_col not in df.columns:
        return f" Column '{item_col}' not found in dataset"

    # تحويل أي قيمة في العمود إلى
    # array<string>
    df_items = df.select(array(col(item_col).cast(StringType())).alias("items")).dropna()

    # FPGrowth
    fp = FPGrowth(
        itemsCol="items",
        minSupport=0.02,
        minConfidence=0.5
    )

    model = fp.fit(df_items)

    # هنا حفظ النتائج
    itemsets_path = "/kaggle/working/fpgrowth_itemsets.csv"
    rules_path = "/kaggle/working/fpgrowth_rules.csv"

    model.freqItemsets.toPandas().to_csv(itemsets_path, index=False)
    model.associationRules.toPandas().to_csv(rules_path, index=False)

    return (
        "✅ FPGrowth completed successfully\n"
        f" Itemsets saved to: {itemsets_path}\n"
        f" Rules saved to: {rules_path}"
    )


In [23]:
import time

def load_uploaded_dataset():
    global df

    if not upload.value or len(upload.value) == 0:
        print(" Please upload a dataset first")
        return None

    file_info = upload.value[0]
    uploaded_filename = file_info['name']
    content = file_info['content']

    # رسالة بسيطة للمستخدم
    print(f" Uploading file '{uploaded_filename}'... please wait")

    start_time = time.time()  # بدء الوقت

    temp_path = f"/kaggle/working/{uploaded_filename}"
    with open(temp_path, "wb") as f:
        f.write(content)

    df = spark.read.csv(temp_path, header=True, inferSchema=True)
    if "_c0" in df.columns:
        df = df.drop("_c0")

    elapsed = time.time() - start_time
    print(f" Dataset uploaded and loaded: {df.count()} rows, {len(df.columns)} columns")
    print(f" Upload + load took {elapsed:.2f} seconds")

    return df


In [24]:
import ipywidgets as widgets
from IPython.display import display, clear_output
from pyspark.sql.functions import col, isnan, when, count, mean, min, max, stddev
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.clustering import KMeans
from pyspark.ml.fpm import FPGrowth
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, RegressionEvaluator

# -------------------------------
# 1️ Widgets
# -------------------------------
user_id = widgets.Text(description="User ID:")

upload = widgets.FileUpload(
    accept='.csv',
    multiple=False,
    description='Upload Dataset'
)

task_type = widgets.Dropdown(
    options=["Descriptive Statistics", "Machine Learning"],
    description="Task:"
)

operation = widgets.Dropdown(description="Operation:")

# Dropdown للعمود الهدف يظهر فقط عند اختيار عمليات ML
target_column = widgets.Dropdown(description="Target Column:")
target_column.layout.display = 'none'  # مخفي بالبداية

run_button = widgets.Button(description="Run", button_style="success")
output = widgets.Output()

# -------------------------------
#  تحديث خيارات العملية بناءً على المهمة
# -------------------------------
def update_operations(*args):
    if task_type.value == "Descriptive Statistics":
        operation.options = [
            "Dataset Info",
            "Missing Values",
            "Descriptive Statistics",
            "Average Numeric Columns"
        ]
        target_column.layout.display = 'none'
    elif task_type.value == "Machine Learning":
        operation.options = [
            "Decision Tree Classification",
            "KMeans Clustering",
            "FPGrowth Frequent Itemsets",
            "Linear Regression"
        ]
       

task_type.observe(update_operations, 'value')



def update_target_column(*args):
    
    if operation.value not in ["Decision Tree Classification", "Linear Regression"]:
        target_column.layout.display = 'none'
        return

 
    target_column.layout.display = 'block'

    # لو الداتا لسه مش مرفوعة
    if df is None:
        target_column.options = []
        return

    # Decision Tree  أي عمود
    if operation.value == "Decision Tree Classification":
        target_column.options = list(df.columns)

    # Linear Regression  أعمدة رقمية فقط
    elif operation.value == "Linear Regression":
        numeric_cols = [
            c for c, t in df.dtypes
            if t in ('int', 'double', 'float', 'long')
        ]
        target_column.options = numeric_cols

operation.observe(update_target_column, 'value')


display(user_id, upload, task_type, operation, target_column, run_button, output)


Text(value='', description='User ID:')

FileUpload(value=(), accept='.csv', description='Upload Dataset')

Dropdown(description='Task:', options=('Descriptive Statistics', 'Machine Learning'), value='Descriptive Stati…

Dropdown(description='Operation:', options=(), value=None)

Dropdown(description='Target Column:', layout=Layout(display='none'), options=(), value=None)

Button(button_style='success', description='Run', style=ButtonStyle())

Output()

In [25]:
upload.value

()

In [26]:
import os
import pandas as pd

RESULTS_DIR = "/kaggle/working/results/"
os.makedirs(RESULTS_DIR, exist_ok=True)

def save_result(df_result, operation_name):
   
    filename = f"{RESULTS_DIR}{operation_name.replace(' ', '_')}.csv"
    
    if 'pandas' in str(type(df_result)):
        df_result.to_csv(filename, index=False)
    else:
        # Spark DataFrame
        df_result.write.mode("overwrite").option("header", True).csv(filename)
    
    print(f" Result of '{operation_name}' saved to {filename}")
    return filename

def run_processing(b):
    with output:
        clear_output()
        
        if not upload.value:
            print(" Please upload a dataset first")
            return
        
        # تحميل الملف
        filename = load_uploaded_dataset()
        print(f" Dataset uploaded and loaded: {filename}")
       
        
        #  تنفيذ العملية
        if task_type.value == "Descriptive Statistics":
            if operation.value == "Dataset Info":
                result = pd.DataFrame([dataset_info()], columns=["Info"])
                print(result.iloc[0,0])
                save_result(result, operation.value)
            elif operation.value == "Missing Values":
                result = missing_values()
                display(result)
                save_result(result, operation.value)
            elif operation.value == "Descriptive Statistics":
                result = descriptive_stats()
                display(result)
                save_result(result, operation.value)
            elif operation.value == "Average Numeric Columns":
                result = average_numeric_columns()
                display(result)
                save_result(result, operation.value)
        
        elif task_type.value == "Machine Learning":
            if operation.value == "Decision Tree Classification":
                 print(run_decision_tree(df, target_column.value))
                # result = run_decision_tree(df)
                # print(result)


            elif operation.value == "Linear Regression":
                result = run_linear_regression(df, target_column.value)
                print(result)

            elif operation.value == "KMeans Clustering":
                print(run_kmeans(df))

            elif operation.value == "FPGrowth Frequent Itemsets":
                result = run_fpgrowth(df, target_column.value)
                print(result)


run_button.on_click(run_processing)


عشان نحذف الأعمدة المعدومة أو الفارغة

I remove the column (_c0) from the dataset.
After inspecting the schema in the previous step, I realized that this column
does not contain meaningful information and most likely represents an index
generated during the CSV loading process.

Keeping such a column would not add any value to the analysis and could
negatively affect performance when processing large datasets.
By removing it early, I reduce the dataset size and ensure that only relevant
features are kept for descriptive statistics and machine learning tasks.

After this cleaning step, the dataset is better prepared for further analysis,
and the next step is to verify the updated structure and continue with
descriptive statistics.

جملة برنت بايثون عادية تحتوي على دالة لين لتحسب عدد الاعمدة وجواها السبارك دوت متغير السبارك الي استخدمناه فوق الي تحتوي على بيانات الداتاسيت لنجيب الاعمدة من الداتا فريم 
وتحته استخدمنا دالة السبارك لطبع السكيما تاعت الداتا فريم زي اسماء الأعمدة ونوعها كسترينج أو انتجر و اذا يقبل العمود قيمة نلل أو لا
و
لنتأكد من عدد الأعمدة أنه تغيير بعد التعديل وحذف العمود الذي ليس له قية فبتلاحظ عدد الأعمدة صار بدل 46 بدل 47

وهنا شرح مجرد دون زيادة لما فعلته باللغة الانجليزية 

To validate data types and ensure compatibility with (Spark MLlib) before performing distributed machine learning tasks .

I check the dataset after removing the (_c0) column.
First, I print the number of columns to confirm that one column has been removed,
and now the dataset has 46 columns instead of 47.

Then, I print the schema of the DataFrame again to inspect the structure.
This helps me verify that all remaining columns are correctly typed
(numerical columns as integers or doubles, categorical columns as strings)
and that the dataset is ready for further analysis.

By doing this, I ensure that no unintended columns remain and that the data
types are correct for Spark MLlib processing. The next step after this is
to start calculating descriptive statistics and understanding the content
of each column for our machine learning tasks.


ما تقلق راح اعدل الكود و التعليقات بس حبيت اجربه واتأكد اذا راح يشتغل او والحمدلله اشتغل كامل

In [1]:

import time
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import StringIndexer, VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

DATASET_PATH = "/kaggle/input/ddos-botnet-attack-on-iot-devices/DDoSdata.csv"

def run_performance_test(num_cores):
    print(f"\n{'='*60}")
    print(f"Running Decision Tree with {num_cores} core(s)")
    print(f"{'='*60}")

    spark = SparkSession.builder \
        .appName(f"DT_{num_cores}_cores") \
        .master(f"local[{num_cores}]") \
        .config("spark.sql.shuffle.partitions", num_cores * 2) \
        .getOrCreate()

    start_time = time.time()

    # Load dataset
    df = spark.read.csv(DATASET_PATH, header=True, inferSchema=True)

    print("📌 Dataset columns:")
    print(df.columns)


    POSSIBLE_LABELS = ["attack", "Attack", "label", "Label", "class", "Category"]

    label_col = None
    for col_name in POSSIBLE_LABELS:
        if col_name in df.columns:
            label_col = col_name
            break

    if label_col is None:
        raise Exception(" No valid label column found in dataset")

    print(f" Using label column: {label_col}")

    # Encode label
    indexer = StringIndexer(
        inputCol=label_col,
        outputCol="label",
        handleInvalid="skip"
    )
    df = indexer.fit(df).transform(df)

    # Numeric features only
    feature_cols = [
        c for c, t in df.dtypes
        if t in ("int", "double", "float", "long") and c != label_col
    ]

    assembler = VectorAssembler(
        inputCols=feature_cols,
        outputCol="features",
        handleInvalid="skip"
    )

    df_final = assembler.transform(df).select("features", "label")

    train_df, test_df = df_final.randomSplit([0.7, 0.3], seed=42)

    dt = DecisionTreeClassifier(
        labelCol="label",
        featuresCol="features",
        maxDepth=4
    )

    model = dt.fit(train_df)
    predictions = model.transform(test_df)
    predictions.count()  # force execution

    total_time = time.time() - start_time

    evaluator = MulticlassClassificationEvaluator(
        labelCol="label",
        predictionCol="prediction",
        metricName="accuracy"
    )

    accuracy = evaluator.evaluate(predictions)

    spark.stop()

    return total_time, accuracy


cores_list = [1, 2, 4, 8]
results = []
baseline_time = None

for cores in cores_list:
    exec_time, acc = run_performance_test(cores)

    if baseline_time is None:
        baseline_time = exec_time

    speedup = baseline_time / exec_time
    efficiency = speedup / cores

    results.append({
        "Cores": cores,
        "Execution Time (sec)": round(exec_time, 2),
        "Speedup": round(speedup, 2),
        "Efficiency": round(efficiency, 2),
        "Accuracy": round(acc, 4)
    })

df_results = pd.DataFrame(results)
df_results.to_csv("/kaggle/working/performance_results.csv", index=False)

print("\n PERFORMANCE TEST COMPLETED SUCCESSFULLY")
print(df_results)
print("\n Saved to /kaggle/working/performance_results.csv")



Running Decision Tree with 1 core(s)


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
26/01/11 12:53:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


📌 Dataset columns:
['_c0', 'pkSeqID', 'stime', 'flgs', 'flgs_number', 'proto', 'proto_number', 'saddr', 'sport', 'daddr', 'dport', 'pkts', 'bytes', 'state', 'state_number', 'ltime', 'seq', 'dur', 'mean', 'stddev', 'sum', 'min', 'max', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate', 'TnBPSrcIP', 'TnBPDstIP', 'TnP_PSrcIP', 'TnP_PDstIP', 'TnP_PerProto', 'TnP_Per_Dport', 'AR_P_Proto_P_SrcIP', 'AR_P_Proto_P_DstIP', 'N_IN_Conn_P_DstIP', 'N_IN_Conn_P_SrcIP', 'AR_P_Proto_P_Sport', 'AR_P_Proto_P_Dport', 'Pkts_P_State_P_Protocol_P_DestIP', 'Pkts_P_State_P_Protocol_P_SrcIP', 'attack', 'category', 'subcategory']
✅ Using label column: attack


26/01/11 12:54:41 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , pkSeqID, stime, flgs_number, proto_number, pkts, bytes, state_number, ltime, seq, dur, mean, stddev, sum, min, max, spkts, dpkts, sbytes, dbytes, rate, srate, drate, TnBPSrcIP, TnBPDstIP, TnP_PSrcIP, TnP_PDstIP, TnP_PerProto, TnP_Per_Dport, AR_P_Proto_P_SrcIP, AR_P_Proto_P_DstIP, N_IN_Conn_P_DstIP, N_IN_Conn_P_SrcIP, AR_P_Proto_P_Sport, AR_P_Proto_P_Dport, Pkts_P_State_P_Protocol_P_DestIP, Pkts_P_State_P_Protocol_P_SrcIP, attack
 Schema: _c0, pkSeqID, stime, flgs_number, proto_number, pkts, bytes, state_number, ltime, seq, dur, mean, stddev, sum, min, max, spkts, dpkts, sbytes, dbytes, rate, srate, drate, TnBPSrcIP, TnBPDstIP, TnP_PSrcIP, TnP_PDstIP, TnP_PerProto, TnP_Per_Dport, AR_P_Proto_P_SrcIP, AR_P_Proto_P_DstIP, N_IN_Conn_P_DstIP, N_IN_Conn_P_SrcIP, AR_P_Proto_P_Sport, AR_P_Proto_P_Dport, Pkts_P_State_P_Protocol_P_DestIP, Pkts_P_State_P_Protocol_P_SrcIP, attack
Expected: _c0 but found: 


Running Decision Tree with 2 core(s)


...

26/01/11 12:59:00 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , pkSeqID, stime, flgs_number, proto_number, pkts, bytes, state_number, ltime, seq, dur, mean, stddev, sum, min, max, spkts, dpkts, sbytes, dbytes, rate, srate, drate, TnBPSrcIP, TnBPDstIP, TnP_PSrcIP, TnP_PDstIP, TnP_PerProto, TnP_Per_Dport, AR_P_Proto_P_SrcIP, AR_P_Proto_P_DstIP, N_IN_Conn_P_DstIP, N_IN_Conn_P_SrcIP, AR_P_Proto_P_Sport, AR_P_Proto_P_Dport, Pkts_P_State_P_Protocol_P_DestIP, Pkts_P_State_P_Protocol_P_SrcIP, attack
 Schema: _c0, pkSeqID, stime, flgs_number, proto_number, pkts, bytes, state_number, ltime, seq, dur, mean, stddev, sum, min, max, spkts, dpkts, sbytes, dbytes, rate, srate, drate, TnBPSrcIP, TnBPDstIP, TnP_PSrcIP, TnP_PDstIP, TnP_PerProto, TnP_Per_Dport, AR_P_Proto_P_SrcIP, AR_P_Proto_P_DstIP, N_IN_Conn_P_DstIP, N_IN_Conn_P_SrcIP, AR_P_Proto_P_Sport, AR_P_Proto_P_Dport, Pkts_P_State_P_Protocol_P_DestIP, Pkts_P_State_P_Protocol_P_SrcIP, attack
Expected: _c0 but found: 


Running Decision Tree with 8 core(s)


📌 Dataset columns:
['_c0', 'pkSeqID', 'stime', 'flgs', 'flgs_number', 'proto', 'proto_number', 'saddr', 'sport', 'daddr', 'dport', 'pkts', 'bytes', 'state', 'state_number', 'ltime', 'seq', 'dur', 'mean', 'stddev', 'sum', 'min', 'max', 'spkts', 'dpkts', 'sbytes', 'dbytes', 'rate', 'srate', 'drate', 'TnBPSrcIP', 'TnBPDstIP', 'TnP_PSrcIP', 'TnP_PDstIP', 'TnP_PerProto', 'TnP_Per_Dport', 'AR_P_Proto_P_SrcIP', 'AR_P_Proto_P_DstIP', 'N_IN_Conn_P_DstIP', 'N_IN_Conn_P_SrcIP', 'AR_P_Proto_P_Sport', 'AR_P_Proto_P_Dport', 'Pkts_P_State_P_Protocol_P_DestIP', 'Pkts_P_State_P_Protocol_P_SrcIP', 'attack', 'category', 'subcategory']
✅ Using label column: attack


26/01/11 13:00:43 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , pkSeqID, stime, flgs_number, proto_number, pkts, bytes, state_number, ltime, seq, dur, mean, stddev, sum, min, max, spkts, dpkts, sbytes, dbytes, rate, srate, drate, TnBPSrcIP, TnBPDstIP, TnP_PSrcIP, TnP_PDstIP, TnP_PerProto, TnP_Per_Dport, AR_P_Proto_P_SrcIP, AR_P_Proto_P_DstIP, N_IN_Conn_P_DstIP, N_IN_Conn_P_SrcIP, AR_P_Proto_P_Sport, AR_P_Proto_P_Dport, Pkts_P_State_P_Protocol_P_DestIP, Pkts_P_State_P_Protocol_P_SrcIP, attack
 Schema: _c0, pkSeqID, stime, flgs_number, proto_number, pkts, bytes, state_number, ltime, seq, dur, mean, stddev, sum, min, max, spkts, dpkts, sbytes, dbytes, rate, srate, drate, TnBPSrcIP, TnBPDstIP, TnP_PSrcIP, TnP_PDstIP, TnP_PerProto, TnP_Per_Dport, AR_P_Proto_P_SrcIP, AR_P_Proto_P_DstIP, N_IN_Conn_P_DstIP, N_IN_Conn_P_SrcIP, AR_P_Proto_P_Sport, AR_P_Proto_P_Dport, Pkts_P_State_P_Protocol_P_DestIP, Pkts_P_State_P_Protocol_P_SrcIP, attack
Expected: _c0 but found: 


✅ PERFORMANCE TEST COMPLETED SUCCESSFULLY
   Cores  Execution Time (sec)  Speedup  Efficiency  Accuracy
0      1                156.76     1.00        1.00       1.0
1      2                 90.14     1.74        0.87       1.0
2      4                 85.89     1.83        0.46       1.0
3      8                 81.84     1.92        0.24       1.0

📁 Saved to /kaggle/working/performance_results.csv
